# Install Detectron2 dependencies

In [1]:
from google.colab import drive
drive.mount('/content/dinhtu')

Mounted at /content/dinhtu


In [2]:
# Install dependencies: use cu101 
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 2.8MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 15.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=2d90b3605e910b4caa8b56c07107c730c8ceefa837362eaca36c718c05b72ee1
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip

In [3]:
# Install detectron2 lastest version:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2MB 718kB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201218-cp36-none-any.whl size=40701 sha256=3f168bf7d3148b14bb476ff648d9cc4cb00cfa5d0541347748456d0af13b6dd0
  Stored in directory: /root/.cache/pip/wheels/b5/83/3f/e74a72d264351b5ada512da845a5c6abb81a34f6333c3a54ce
  Created wheel for iopath: filename=iopath-0.1.2-cp36-none-any.whl size=10508 sha256=c4bba031f563df4758f7d437f46c5f52598ddd2a9c49e6c9e4a8139aab1ebc7b
  Stored in directory: /root/.cache/pip/wheels/9e/01/e4/1b68f5a2a6b9450ea4246d91840a77e1169f7d4722d76bbc47
Successfully built fvcore iopath


In [4]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common libraries
import numpy as np
import cv2
import tqdm
import random
from google.colab.patches import cv2_imshow

# Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
import time

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# Demo inference with Detectron2 saved weights



In [5]:
%mkdir output
%cd output
!unzip "/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/pretrained_model_r101_dc5.zip"
%cd ..

/content/output
Archive:  /content/dinhtu/MyDrive/vehicles_detection/Tu_workspace/pretrained_model_r101_dc5.zip
  inflating: instances_predictions.pth  
  inflating: model_final.pth         
  inflating: anotations_coco_demo.json  
  inflating: coco_instances_results.json  
  inflating: metrics.json            
  inflating: events.out.tfevents.1607447348.754d2414cc37.57.0  
  inflating: config.yaml             
  inflating: last_checkpoint         
/content


In [6]:
import json
from detectron2.data import MetadataCatalog 
with open('/content/output/anotations_coco_demo.json') as f:
  data = json.load(f)

name_classes=(data['classes_pretrained'])
model_name=(data['dataset_model_name'])
#Set up my classes to predict on data demo
MetadataCatalog.get(model_name).thing_classes = name_classes


In [7]:
print(name_classes,model_name)

['vehicles', 'bus', 'car', 'container', 'motorcycle', 'truck'] coco_data_test


In [8]:
import os
#Running pretrained model
cfg = get_cfg()
cfg.merge_from_file("/content/output/config.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)
demo_metadata = MetadataCatalog.get(model_name)
print(demo_metadata)

Metadata(name='coco_data_test', thing_classes=['vehicles', 'bus', 'car', 'container', 'motorcycle', 'truck'])


#Demo on a image

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
im = cv2.imread("/content/demo1.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],metadata=demo_metadata,scale=0.8)
out_one = v.draw_instance_predictions(outputs["instances"].to("cpu"))
out_one.save('/content/demo.png')
cv2_imshow(out_one.get_image()[:, :, ::-1])

#Demo on entire image in folder extracted frame on my raw video 

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
import time

#save_img_predicted = os.mkdir('/demo03_predicted')
#for i in [3,5,10,22]:
j=0
start_time = time.time()
for imageName in glob.glob('/content/dinhtu/MyDrive/vehicles_detection/demo/*png'):
    j+=1
    im = cv2.imread(imageName)
    outputs = predictor(im)
    #Visulizer imagme predicted
    v = Visualizer(im[:, :, ::-1],metadata=demo_metadata, scale=0.8)
    out_entire = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #Save predicted
    #out_entire.save('/content/demo_predicted_{}/{}.png'.format(i,j))
    print(j)
    #cv2.imwrite(save_img_predicted, out_entire.get_image()[:, :, ::-1])
    #Show predicted
    cv2_imshow(out_entire.get_image()[:, :, ::-1])
end_time = time.time()
total_time=end_time-start_time
print("Total time:",total_time)
print("Total images:",j)
#print("Average time:",total_time/count)

In [ ]:
#Convert PNG to JPG
from PIL import Image
for i in [3,5,10,22]:
  for j in range(500,1500):
    im = Image.open('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace_2/save_img_demo/demo_predicted_{}/{}.png'.format(i,j))
    rgb_im = im.convert('RGB')
    rgb_im.save('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace_2/save_img_demo/demo_predicted_0{}/{}.jpg'.format(i,j))

#Demo on my raw video and save video predicted

In [ ]:
for i in [3,5,10,22]:
  # Extract video properties
  video = cv2.VideoCapture('/content/dinhtu/MyDrive/vehicles_detection/train/{}.mp4'.format(i))
  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frames_per_second = video.get(cv2.CAP_PROP_FPS)
  num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

  # Initialize video writer
  video_writer = cv2.VideoWriter('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace_2/video_demo/demo_final_{}.mp4'.format(i), fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

  # Initialize visualizer
  #v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)
  v_video = VideoVisualizer(demo_metadata, ColorMode.IMAGE)
  def runOnVideo(video):
      """ Runs the predictor on every frame in the video (unless maxFrames is given),
      and returns the frame with the predictions drawn.
      """

      j = 500
      while (video.isOpened()):
          ret, frame = video.read()
          if ret == False or j==1500:
              break
          if j>=500:
             cv2.imwrite('{}_to_demo.jpg'.format(i),frame)

          # Get prediction results for this frame
          outputs = predictor(frame)

          # Make sure the frame is colored
          frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

          # Draw a visualization of the predictions using the video visualizer
          visualization = v_video.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

          # Convert Matplotlib RGB format to OpenCV BGR format
          visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

          yield visualization

          j += 1
          #if j > maxFrames:
              #break

  # Create a cut-off for debugging
  num_frames = 1000

  # Enumerate the frames of the video
  #for j in range(500,1500):
  for visualization in tqdm.tqdm(runOnVideo(video), total=num_frames):
      #Write demo image
      cv2.imwrite('{}_demo.png'.format(i),visualization)
      # Write to video file
      video_writer.write(visualization)
  # Release resources
  video.release()
  video_writer.release()
  cv2.destroyAllWindows()

100%|██████████| 1000/1000 [09:19<00:00,  1.79it/s]


In [ ]:
for i in [3,5,10,22]:
  # Extract video properties
  video = cv2.VideoCapture('/content/dinhtu/MyDrive/vehicles_detection/train/{}.mp4'.format(i))
  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frames_per_second = video.get(cv2.CAP_PROP_FPS)
  num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

  # Initialize video writer
  video_writer = cv2.VideoWriter('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace_2/video_demo/demo_final_{}.mp4'.format(i), fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

  # Initialize visualizer
  #v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)
  v_video = VideoVisualizer(demo_metadata, ColorMode.IMAGE)

  # Load 500->1000 frames extracted
  for imageName in glob.glob('/content/dinhtu/MyDrive/vehicles_detection/Tu_workspace_2/input/frames_{}/*jpg'.format(i)):
      frame = cv2.imread(imageName)
  #Runs the predictor on every frame in the video (unless maxFrames is given),and returns the frame with the predictions drawn.
  # Get prediction results for this frame
  outputs = predictor(frame)

  # Make sure the frame is colored
  frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

  # Draw a visualization of the predictions using the video visualizer
  visualizations = v_video.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

  # Convert Matplotlib RGB format to OpenCV BGR format
  visualizations = cv2.cvtColor(visualizations.get_image(), cv2.COLOR_RGB2BGR)

  #yield visualizations
  #if j > maxFrames:
  #break

  # Create a cut-off for debugging
  num_frames = 1000

  # Enumerate the frames of the video
  #for j in range(500,1500):
  for visualization in tqdm.tqdm(visualizations, total=num_frames):
      #Write demo image
      cv2.imwrite('{}_demo.png'.format(i),visualization)
      # Write to video file
      video_writer.write(visualization)
  # Release resources
  video.release()
  video_writer.release()
  cv2.destroyAllWindows()